# Atomization 

For the source code, see [atomization_energy](https://workgraph-collections.readthedocs.io/en/latest/ase/espresso/module.html#workgraph_collections.ase.espresso.atomization_energy.atomization_energy).


## Introduction
The atomization energy, $\Delta E$, of a molecule can be expressed as:

$$
\Delta E = n_{\text{atom}} \times E_{\text{atom}} - E_{\text{molecule}}
$$

Where:

- $\Delta E$ is the atomization energy of the molecule.
- $n_{\text{atom}}$ is the number of atoms.
- $E_{\text{atom}}$ is the energy of an isolated atom.
- $E_{\text{molecule}}$ is the energy of the molecule.

## Visualizing the WorkGraph Builder


In [1]:
from workgraph_collections.ase.espresso.atomization_energy import atomization_energy

node = atomization_energy.node()
node.to_html()

## Visualizing the WorkGraph

In [2]:
from workgraph_collections.ase.espresso.atomization_energy import atomization_energy

wg = atomization_energy()
wg.to_html()

## Example: Atomization energy of a molecule N2

### Prepare the inputs and submit the workflow


In [3]:
from ase.build import molecule
from ase import Atoms
from aiida import load_profile
from workgraph_collections.ase.espresso.atomization_energy import atomization_energy

load_profile()

# create input structure node
n_atom = Atoms("N", pbc=True)
n_atom.center(vacuum=5.0)
n2_molecule = molecule("N2", pbc=True)
n2_molecule.center(vacuum=5.0)

metadata = {
    "options": {
        'prepend_text' : """eval "$(conda shell.posix hook)"
        conda activate aiida
        export OMP_NUM_THREADS=1
        """,
    }
}
pseudopotentials = {"N": "N.pbe-n-rrkjus_psl.1.0.0.UPF"}
# pseudo_dir = "/home/xing/data/ase/espresso_pseudo"
pseudo_dir = "/home/wang_x3/datas/pseudos/psl"
input_data = {
    "system": {"ecutwfc": 30, "ecutrho": 240,
               "occupations": "smearing",
                "degauss": 0.01,
                "smearing": "cold",},
}
#------------------------- Set the inputs -------------------------
wg = atomization_energy()
wg.nodes["scf_atom"].set({"atoms": n_atom,
                          "pseudopotentials": pseudopotentials,
                          "pseudo_dir": pseudo_dir,
                          "input_data": input_data,
                          "computer": "localhost",
                          "metadata": metadata})
wg.nodes["scf_mol"].set({"atoms": n2_molecule,
                          "pseudopotentials": pseudopotentials,
                          "pseudo_dir": pseudo_dir,
                          "input_data": input_data,
                          "computer": "localhost",
                          "metadata": metadata})
wg.nodes["calc_atomization_energy"].set({"molecule": n2_molecule, "computer": "localhost"})
#------------------------- Submit the calculation -------------------
# wg.run()
wg.submit(wait=True, timeout=200)
#------------------------- Print the output -------------------------
print('Energy of a N atom:                  {:0.3f}'.format(wg.nodes['scf_atom'].outputs["results"].value.value['energy']))
print('Energy of an un-relaxed N2 molecule: {:0.3f}'.format(wg.nodes['scf_mol'].outputs["results"].value.value['energy']))
print('Atomization energy:                  {:0.3f} eV'.format(wg.nodes['calc_atomization_energy'].outputs["result"].value.value))



WorkGraph node created, PK: 16469
Energy of a N atom:                  -269.881
Energy of an un-relaxed N2 molecule: -556.008
Atomization energy:                  16.246 eV
